<a href="https://colab.research.google.com/github/rojinadeuja/NLP-Model-Implementations/blob/master/Word2vec-using-Custom-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Modules

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split

## Load Custom model

In [3]:
from gensim.models import KeyedVectors

# Load custom vectors directly from the file
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/en_imdb_word2vec_300.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
# Vocabulary size of the custom model
len(model.vocab)

11727

In [5]:
# Check similarity between words in model
model.similarity('simple', 'impossible')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.16050674

## Load Datatset

In [6]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/IMDB.csv')
df.replace(['positive', 'negative'], [1, 0], inplace=True)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


## Data Pre-processing

In [7]:
def preprocess(s):
    '''Function for data pre-processing'''
    # Removing html tags
    TAG_RE = re.compile(r'<[^>]+>')
    sentence = TAG_RE.sub('', s)
    
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

## Create X and y matrices

In [8]:
X = []
sentences = list(df['review'])
for sentence in sentences:
    X.append(preprocess(sentence))

y = df['sentiment']

## Split dataset into train and test

In [9]:
# Create train-test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))

40000 10000 40000 10000


In [10]:
# Convert into numpy arrays for processing with tensorflo
y_train = np.array(y_train)
y_test = np.array(y_test)
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'list'> <class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


## Tokenize the data

In [11]:
# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [12]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 300

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## Create Embeddings

In [13]:
embedding_matrix = np.zeros((len(model.wv.vocab), 300)) # vector_dim =300
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


## Classification using a simple Neural Network

In [14]:
# Text Classification with a Simple Neural Network
vocab_size = len(model.wv.vocab)
model = Sequential()
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          3518100   
_________________________________________________________________
flatten (Flatten)            (None, 90000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 90001     
Total params: 3,608,101
Trainable params: 90,001
Non-trainable params: 3,518,100
_________________________________________________________________
None


## Train the model

In [16]:
# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 10s 39ms/step - loss: 0.7134 - acc: 0.5698 - val_loss: 0.6515 - val_acc: 0.6279
Epoch 2/10
250/250 [==============================] - 10s 40ms/step - loss: 0.4396 - acc: 0.8161 - val_loss: 0.6676 - val_acc: 0.6244
Epoch 3/10
250/250 [==============================] - 9s 35ms/step - loss: 0.3315 - acc: 0.8867 - val_loss: 0.6789 - val_acc: 0.6352
Epoch 4/10
250/250 [==============================] - 9s 35ms/step - loss: 0.2614 - acc: 0.9267 - val_loss: 0.6810 - val_acc: 0.6463
Epoch 5/10
250/250 [==============================] - 10s 38ms/step - loss: 0.2136 - acc: 0.9510 - val_loss: 0.7058 - val_acc: 0.6440
Epoch 6/10
250/250 [==============================] - 10s 39ms/step - loss: 0.1763 - acc: 0.9665 - val_loss: 0.7800 - val_acc: 0.6270
Epoch 7/10
250/250 [==============================] - 9s 36ms/step - loss: 0.1497 - acc: 0.9766 - val_loss: 0.7639 - val_acc: 0.6419
Epoch 8/10
250/250 [==============================] - 9s 37ms/ste

## Evaluate model on test data

In [17]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=1)
print("\nTest Accuracy:", score[1])

313/313 [==============================] - 2s 8ms/step - loss: 0.8390 - acc: 0.6444

Test Accuracy: 0.6444000005722046
